# End-to-End VM Connectivity Across Subnets Using `sshuttle`, WireGuard, and Static Routing

This network slice consists of three Virtual Machines (VMs) connected across two subnets, forming a routed topology:

* **Node1 - Node2 LAN**: `192.168.1.0/24`
* **Node2 - Node3 WAN**: `192.168.2.0/24`

---

## **Routing Configuration Options**

### 1. `sshuttle` Tunneling (Automated TCP Forwarding over SSH)

As a simpler alternative to static routing, this notebook demonstrates [`sshuttle`](https://github.com/sshuttle/sshuttle), a tool that transparently forwards TCP traffic over SSH.

* Run on **Node1**, it tunnels traffic via SSH to **Node2**
* All packets to `192.168.2.0/24` or `192.168.3.0/24` are automatically forwarded
* No static routes or IP forwarding setup needed on Node1
* Ideal for fast setup, temporary debugging, or tunneling across firewalled segments

---

### 2. WireGuard Tunneling (Encrypted L3 Overlay)

This notebook also demonstrates [**WireGuard**](https://www.wireguard.com/), a lightweight, modern VPN protocol for creating encrypted point-to-point tunnels.

* **Node1**, **Node2**, and **Node3** form a WireGuard overlay using interface `wg0`
* Peers are connected via internal tunnel IPs in the `10.0.0.0/24` subnet
* WireGuard routes traffic securely between `192.168.1.0/24` and `192.168.2.0/24` through Node2
* Requires enabling IP forwarding and configuring `iptables` for routing between underlay and overlay

**Benefits:**

* Full encryption of all traffic (TCP, UDP, ICMP)
* Minimal configuration and strong performance
* Customizable per-peer routing via `AllowedIPs`
* Reliable over NAT/firewalled environments using `PersistentKeepalive`

This option is ideal for **secure overlay networks**, especially in research, educational, or partially trusted environments like **FABRIC**.

---

### 3. Static Routing (Manual Configuration)

* **Node2** manually routes traffic between `192.168.1.0/24` and `192.168.2.0/24`
* Routes are set up using `ip route` and `sysctl` to enable forwarding
* **Node3** can be similarly configured if extending beyond two hops
* No encryption, but provides transparency and control over routing behavior

This method is useful for understanding traditional IP routing principles and debugging layered topologies in testbeds.

## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

In [ ]:
slice_name = 'MySlice-tunnels-static-routes'
[site1,site2, site3]  = fablib.get_random_sites(count=3)
#[site1,site2, site3] = ["SALT", "STAR", "LOSA"]
print(f"Sites: {site1}, {site2}, {site3}")

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

net1_name ='net1'
net2_name='net2'
net3_name='net3'

net1_subnet = "192.168.1.0/24"
net2_subnet = "192.168.2.0/24"
net3_subnet = "192.168.3.0/24"
image = "default_ubuntu_24"

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name=net1_name, subnet=IPv4Network(net1_subnet))
net2 = slice.add_l2network(name=net2_name, subnet=IPv4Network(net2_subnet))

# Node1
node1 = slice.add_node(name=node1_name, site=site1, image=image)
iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)

# Node2
node2 = slice.add_node(name=node2_name, site=site2, image=image)
iface2 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface2.set_mode('auto')
net1.add_interface(iface2)

iface3 = node2.add_component(model='NIC_Basic', name='nic2').get_interfaces()[0]
iface3.set_mode('auto')
net2.add_interface(iface3)

# Node3
node3 = slice.add_node(name=node3_name, site=site3, image=image)
iface4 = node3.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface4.set_mode('auto')
net2.add_interface(iface4)

#Submit Slice Request
slice.submit()

## Verify Connectivity

Before proceeding with routing or tunneling setup, ensure that basic IP-layer connectivity exists between directly connected nodes:

- Check that **Node1 and Node2** are reachable from each other
- Check that **Node2 and Node3** are reachable from each other

In [ ]:
# Check ping on Site1 to Site2 (L2STS)
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=net1_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

In [ ]:
# Check ping on Site2 to Site3 (L2STS)
slice = fablib.get_slice(slice_name)

node2 = slice.get_node(name=node2_name)        
node3 = slice.get_node(name=node3_name)           

node3_addr = node3.get_interface(network_name=net2_name).get_ip_addr()

stdout, stderr = node2.execute(f'ping -c 5 {node3_addr}')

## Configure SSH Key-Based Access

Generate SSH key pairs for both the `ubuntu` and `root` users.  
Distribute the corresponding public keys to all nodes by appending them to the appropriate `authorized_keys` files, enabling passwordless SSH access between nodes.


In [ ]:
for n in slice.get_nodes():
    n.execute('ssh-keygen -t rsa -N "" -f /home/ubuntu/.ssh/id_rsa', quiet=True)
    n.execute('sudo ssh-keygen -t rsa -N "" -f /root/.ssh/id_rsa', quiet=True)

In [ ]:
keys = {}
# Step 1: Collect public keys from each node
for n in slice.get_nodes():
    ubuntu_key, _ = n.execute("cat /home/ubuntu/.ssh/id_rsa.pub", quiet=True)
    root_key, _ = n.execute("sudo cat /root/.ssh/id_rsa.pub", quiet=True)
    keys[n.get_name()] = {
        "ubuntu": ubuntu_key.strip(),
        "root": root_key.strip()
    }

# Step 2: Distribute public keys to all other nodes
for n in slice.get_nodes():
    for other_node_name, node_keys in keys.items():
        if other_node_name == n.get_name():
            continue
        n.execute(f'echo "{node_keys["ubuntu"]}" >> /home/ubuntu/.ssh/authorized_keys')
        n.execute(f'sudo sh -c \'echo "{node_keys["root"]}" >> /root/.ssh/authorized_keys\'')


## Option 1: `sshuttle` — Simplified Tunneling Over SSH

[`sshuttle`](https://github.com/sshuttle/sshuttle) provides a transparent, TCP-based VPN-like tunnel using only SSH. This option simplifies cross-node connectivity by automatically forwarding traffic from one node to a remote subnet without requiring static routes or manual IP forwarding setup.

In this configuration:

- `sshuttle` runs on **Node1** and creates an SSH tunnel to **Node2**
- Any TCP traffic destined for the `192.168.2.0/24` subnets is captured and forwarded through this tunnel
- No modifications are needed on **Node3** or beyond, as long as Node2 can reach them

### Why Use `sshuttle`?
- You want a quick and minimal setup
- You do not have `sudo` access on all nodes to configure IP forwarding
- You're working in a firewalled or restricted environment

> ⚠️ Note: `sshuttle` only supports TCP traffic (e.g., SSH, HTTP) — not ICMP (`ping`) or UDP-based protocols.


In [ ]:
slice = fablib.get_slice(slice_name)
node1 = slice.get_node(name=node1_name)  
node2 = slice.get_node(name=node2_name)        
node3 = slice.get_node(name=node3_name)

In [ ]:
node1_net1_addr = node1.get_interface(network_name=net1_name).get_ip_addr()
node2_net1_addr = node2.get_interface(network_name=net1_name).get_ip_addr()

node2_net2_addr = node2.get_interface(network_name=net2_name).get_ip_addr()
node3_net2_addr = node3.get_interface(network_name=net2_name).get_ip_addr()

In [ ]:
for n in slice.get_nodes():
    stdout, stderr = n.execute("sudo apt update && sudo apt install -y sshuttle net-tools", quiet=True)

### Start `sshuttle` from Node1 and Add a Dummy Route

To enable dynamic tunneling from **Node1** to the remote subnet via **Node2**, follow these steps:

#### 1. Launch `sshuttle` on Node1

Run the following command with `sudo` to start tunneling TCP traffic for the target subnet:

```bash
sudo sshuttle --method=nat \
  --ssh-cmd 'ssh -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null' \
  -r root@192.168.1.1 \
  192.168.2.0/24 \
  -vv


In [ ]:
node1 = slice.get_node(node1_name)
cmd = f"sudo sshuttle --method=nat   --ssh-cmd 'ssh -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null' -r root@{node2_net1_addr} {net2_subnet} --daemon"
print(f"Executing: {cmd}")
stdout, stderr = node1.execute(cmd)

#### 2. Add a Dummy Route to Force Traffic into iptables
On some systems, the kernel won’t allow packets to be emitted for unreachable networks — which prevents sshuttle from seeing them. To fix this, add a dummy route via loopback:

This route doesn't send traffic to loopback — it simply convinces the kernel to emit packets, allowing iptables to redirect them through sshuttle.

In [ ]:
# Add a dummy route to force traffic to hit iptables
stdout, stderr = node1.execute(f"sudo ip route add {net2_subnet} via 127.0.0.1 dev lo")

#### 3. Test the Tunnel
Send a TCP packet (e.g., to port 22) to confirm redirection:

You should now be able to access Node3 as if directly connected.

In [ ]:
stdout, stderr = node1.execute(f'nc -zv {node3_net2_addr} 22')

## Option 2: WireGuard Tunnel (Encrypted L3 Tunnel)

WireGuard is a modern, lightweight, high-performance VPN protocol suitable for setting up encrypted tunnels across routed networks like FABRIC.

This option demonstrates how to use WireGuard to tunnel traffic between **Node1** and **Node3** via **Node2**, forming an L3 encrypted overlay.

### Why Use WireGuard?

- **End-to-End Encryption**: All traffic between Node1 and Node3 is encrypted.
- **Bypasses Static Routing Complexity**: Tunnel hides underlying subnet structure, reducing the need for route manipulation.
- **Protocol Flexibility**: Supports ICMP, TCP, UDP—unlike sshuttle which is TCP-only.
- **Lightweight and Fast**: Minimal performance overhead, ideal for research environments.

### Topology Overview

- **Underlay Subnets:**
  - Node1 ↔ Node2 (LAN): `192.168.1.0/24`
  - Node2 ↔ Node3 (WAN): `192.168.2.0/24`

- **WireGuard Overlay (Tunnel IPs):**
  - Node1: `10.0.0.1`
  - Node2: `10.0.0.2`
  - Node3: `10.0.0.3`

**Node2** acts as a WireGuard relay for Node1 ↔ Node3, and also bridges overlay-to-underlay with IP forwarding and appropriate `iptables` rules.

### When to Use This Option

- When encrypted inter-node communication is required
- When you want better protocol support than sshuttle
- When route configuration is constrained or complex

In the next sections, we will:
- Install and configure WireGuard on all three nodes
- Establish persistent tunnels
- Verify encrypted ping and connectivity between Node1 and Node3

In [ ]:
slice = fablib.get_slice(slice_name)
node1 = slice.get_node(name=node1_name)  
node2 = slice.get_node(name=node2_name)        
node3 = slice.get_node(name=node3_name)

node1_net1_addr = node1.get_interface(network_name=net1_name).get_ip_addr()
node2_net1_addr = node2.get_interface(network_name=net1_name).get_ip_addr()

node2_net2_ifc = node2.get_interface(network_name=net2_name)
node2_net2_addr = node2_net2_ifc.get_ip_addr()
node2_net2_ifc_name = node2_net2_ifc.get_os_interface()

node3_net2_addr = node3.get_interface(network_name=net2_name).get_ip_addr()

### Cleanup Any Leftover Configs Before Setting Up WireGuard

Before configuring WireGuard, ensure the system is free of conflicting settings that might interfere with the tunnel.

- **Remove any static routes:**

  Run the following on each node as needed:

  ```
  sudo ip route del <ROUTE>
  ```
- **Stop sshuttle**:

If sshuttle was started in daemon mode:
```
sudo pkill -f sshuttle
```

In [ ]:
stdout, stderr = node1.execute(f"sudo ip route del {net2_subnet}")
stdout, stderr = node3.execute(f"sudo ip route del {net1_subnet}")

In [ ]:
stdout, stderr = node1.execute(f"sudo pkill -f sshuttle")
stdout, stderr = node3.execute(f"sudo pkill -f sshuttle")

### 1. Install WireGuard on All Nodes and Generate WireGuard Key Pairs

WireGuard needs to be installed on each of the nodes (**Node1**, **Node2**, and **Node3**) participating in the tunnel.


In [ ]:
keys = {}
for n in slice.get_nodes():
    stdout, stderr = n.execute("sudo apt-get update && sudo apt-get install -y wireguard", quiet=True)
    stdout, stderr = n.execute("wg genkey | tee privatekey | wg pubkey > publickey")
    priv_key, stderr = n.execute("cat privatekey", quiet=True)
    pub_key, stderr = n.execute("cat publickey", quiet=True)
    keys[n.get_name()] = {
        "public": pub_key,
        "private": priv_key,
    }

### 2. Configure WireGuard Interfaces

#### Setup WireGuard on Node1

Create the configuration file `/etc/wireguard/wg0.conf` on **Node1**:

In [ ]:
node1_wg_conf = f"""\
[Interface]
PrivateKey = {keys.get(node1.get_name(), {}).get('private')}
Address = 10.0.0.1/24
ListenPort = 51820

[Peer]
PublicKey = {keys.get(node2.get_name(), {}).get('public')}
Endpoint = {node2_net1_addr}:51820
AllowedIPs = 10.0.0.0/24, {net2_subnet}
PersistentKeepalive = 25
"""

stdout, stderr = node1.execute(f"echo '{node1_wg_conf}' | sudo tee /etc/wireguard/wg0.conf > /dev/null")

stdout, stderr = node1.execute("sudo systemctl enable wg-quick@wg0 && sudo systemctl start wg-quick@wg0")

#### Setup WireGuard on Node2

Create the configuration file `/etc/wireguard/wg0.conf` on **Node2**:

In [ ]:
node2_wg_conf = f"""\
[Interface]
PrivateKey = {keys.get(node2.get_name(), {}).get('private')}
Address = 10.0.0.2/24
ListenPort = 51820
PostUp = iptables -A FORWARD -i %i -j ACCEPT; iptables -A FORWARD -o %i -j ACCEPT; iptables -t nat -A POSTROUTING -o {node2_net2_ifc_name} -j MASQUERADE
PostDown = iptables -D FORWARD -i %i -j ACCEPT; iptables -D FORWARD -o %i -j ACCEPT; iptables -t nat -D POSTROUTING -o {node2_net2_ifc_name} -j MASQUERADE

[Peer]
PublicKey = {keys.get(node1.get_name(), {}).get('public')}
Endpoint = {node1_net1_addr}:51820
AllowedIPs = 10.0.0.1/32
PersistentKeepalive = 25

[Peer]
PublicKey = {keys.get(node3.get_name(), {}).get('public')}
Endpoint = {node3_net2_addr}:51820
AllowedIPs = 10.0.0.3/32
PersistentKeepalive = 25
"""

stdout, stderr = node2.execute(f"echo '{node2_wg_conf}' | sudo tee /etc/wireguard/wg0.conf > /dev/null")
stdout, stderr = node2.execute("sudo systemctl enable wg-quick@wg0 && sudo systemctl start wg-quick@wg0")

#### Setup WireGuard on Node3

Create the configuration file `/etc/wireguard/wg0.conf` on **Node3**:

In [ ]:
node3_wg_conf = f"""\
[Interface]
PrivateKey = {keys.get(node3.get_name(), {}).get('private')}
Address = 10.0.0.3/24
ListenPort = 51820

[Peer]
PublicKey = {keys.get(node2.get_name(), {}).get('public')}
Endpoint = {node2_net2_addr}:51820
AllowedIPs = 10.0.0.0/24, {net1_subnet}
PersistentKeepalive = 25
"""

stdout, stderr = node3.execute(f"echo '{node3_wg_conf}' | sudo tee /etc/wireguard/wg0.conf > /dev/null")
stdout, stderr = node3.execute("sudo systemctl enable wg-quick@wg0 && sudo systemctl start wg-quick@wg0")

### 4. Enable IP Forwarding on Node2

To allow Node2 to route traffic between `Node1` and `Node3` over the WireGuard tunnel, enable IPv4 forwarding:

In [ ]:
stdout, stderr = node2.execute("echo 1 | sudo tee /proc/sys/net/ipv4/ip_forward && sudo sysctl -w net.ipv4.ip_forward=1 && sudo sysctl -p")

### 5. Test Tunnel

Once the WireGuard interfaces are configured and active on all nodes, and IP forwarding is enabled on Node2, you can verify tunnel connectivity.

#### From Node1:
Try pinging Node3’s WireGuard IP or private interface IP:

```
ping 10.0.0.3
```
#### From Node3:
Try pinging Node1’s WireGuard IP or private interface IP:

```
ping 10.0.0.1
```
If the tunnel is working correctly, you should see successful replies. You can also inspect WireGuard status and packet counts with:
```
sudo wg show
```
This confirms end-to-end encrypted L3 connectivity through the WireGuard tunnel.

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 10.0.0.3')
stdout, stderr = node1.execute(f'ping -c 5 {node3_net2_addr}')

In [ ]:
stdout, stderr = node3.execute(f'ping -c 5 10.0.0.1')
stdout, stderr = node3.execute(f'ping -c 5 {node1_net1_addr}')

In [ ]:
stdout, stderr = node1.execute(f'nc -zv {node3_net2_addr} 22')

## Option 3: Static Routes and IP Forwarding

This approach demonstrates how to manually configure static routes and enable IP forwarding to achieve end-to-end connectivity across subnets.

In this setup:

- **Node2** acts as a router between `Node1` and `Node3`, bridging `192.168.1.0/24` and `192.168.2.0/24`
- Each node’s routing table is explicitly updated using `ip route` commands
- IP forwarding is enabled on intermediary nodes to allow packets to be relayed between interfaces

### Why Use Static Routes?
- You want full control and visibility into packet routing
- You’re building realistic network experiments
- You need to support all protocols (TCP, UDP, ICMP, etc.)
- Unlike sshuttle, this approach supports all IP traffic, not just TCP.

In [ ]:
slice = fablib.get_slice(slice_name)
node1 = slice.get_node(name=node1_name)  
node2 = slice.get_node(name=node2_name)        
node3 = slice.get_node(name=node3_name)

node1_net1_addr = node1.get_interface(network_name=net1_name).get_ip_addr()
node2_net1_addr = node2.get_interface(network_name=net1_name).get_ip_addr()

node2_net2_addr = node2.get_interface(network_name=net2_name).get_ip_addr()
node3_net2_addr = node3.get_interface(network_name=net2_name).get_ip_addr()

### 1. Cleanup Any Leftover Configs Before Setting Up Static Routes

Before configuring WireGuard, ensure the system is free of conflicting settings that might interfere with the tunnel.

- **Remove any static routes:**

  Run the following on each node as needed:

  ```
  sudo ip route del <ROUTE>
  ```
- **Stop sshuttle**:

If sshuttle was started in daemon mode:
```
sudo pkill -f sshuttle
```
- **Stop wireguard**:

If sshuttle was started in daemon mode:
```
sudo pkill -f sshuttle
```

In [ ]:
stdout, stderr = node1.execute(f"sudo ip route del {net2_subnet} via 127.0.0.1 dev lo")

2. **Enable IP Forwarding**  
   On **Node2**, enable IP forwarding:
   ```bash
   sudo sysctl -w net.ipv4.ip_forward=1

In [ ]:
stdout, stderr = node2.execute("echo 1 | sudo tee /proc/sys/net/ipv4/ip_forward && sudo sysctl -w net.ipv4.ip_forward=1 && sudo sysctl -p")

3. **Set Up Static Routes**
- Add a static route on Node1 to reach Node3 via Node2
- Add a static route on Node3 to reach Node1 via Node2

In [ ]:
node1_net1_addr = node1.get_interface(network_name=net1_name).get_ip_addr()
node2_net1_addr = node2.get_interface(network_name=net1_name).get_ip_addr()

node2_net2_addr = node2.get_interface(network_name=net2_name).get_ip_addr()
node3_net2_addr = node3.get_interface(network_name=net2_name).get_ip_addr()


print("Setup routes on Node1")
stdout, stderr = node1.execute(f"sudo ip route add {net2_subnet} via {node2_net1_addr}")
stdout, stderr = node1.execute(f"sudo ip route list")

print()
print("Setup routes on Node3")
stdout, stderr = node3.execute(f"sudo ip route add {net1_subnet} via {node2_net2_addr}")
stdout, stderr = node3.execute(f"sudo ip route list")

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 {node3_net2_addr}')

In [ ]:
stdout, stderr = node1.execute(f'nc -zv {node3_net2_addr} 22')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()